<a href="https://colab.research.google.com/github/atrisovic/citation-study/blob/main/Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KeyWord Extraction

Question: What are the top keywords for all the papers?

Answer: Top 10 keywords: [('model', 78186), ('use', 42882), ('training', 28528), ('language', 27349), ('task', 23502), ('train', 20948), ('image', 19266), ('network', 19247), ('dataset', 18421), ('result', 18229)]

Total number of unique keywords assessed: 329163

Papers: 754/754

In [1]:
!pip install requests PyPDF2 tqdm


import pandas as pd
import requests
from tqdm import tqdm
import os
from PyPDF2 import PdfReader
from collections import Counter
import re

import nltk
from nltk.corpus import stopwords

import spacy


# English language model
nlp = spacy.load("en_core_web_sm")

# stopwords to exclude (e.g. the, is, and...)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# biblio words
additional_stopwords = ['et', 'al']

!pip install beautifulsoup4
from bs4 import BeautifulSoup

import logging
logging.getLogger("PyPDF2").setLevel(logging.ERROR)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# create directories: one for storing the downloaded PDFs and another for storing the converted text files

!mkdir pdfs
!mkdir texts

In [3]:
pcd_dataset = pd.read_csv("https://epochai.org/data/epochdb/all_systems.csv")

In [4]:
def get_pdf_url(link):
    if link.endswith(".pdf"):
        return link
    elif "arxiv.org" in link:
        return link.replace("abs", "pdf") + ".pdf"
    elif "nature.com" in link:
        # BeautifulSoup to find the PDF link on Nature pages
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_link = soup.find('a', {'data-track-action': 'download pdf'})
        return pdf_link['href'] if pdf_link else None
    return None

def download_pdfs(dataset, column='Link', save_path='pdfs/'):
    """Download PDFs from the dataset."""
    failed_links = []
    for i, row in tqdm(dataset.iterrows(), total=len(dataset)):
        link = row[column]
        if pd.notnull(link):
            pdf_url = get_pdf_url(link)
            if pdf_url:
                file_name = f"{save_path}{row['System'].replace('/', '_')}.pdf"
                try:
                    response = requests.get(pdf_url)
                    if response.status_code == 200:
                        with open(file_name, 'wb') as f:
                            f.write(response.content)
                    else:
                        failed_links.append(pdf_url)
                except Exception as e:
                    print(f"Error downloading {pdf_url}: {e}")
                    failed_links.append(pdf_url)
            else:
                failed_links.append(link)

    if failed_links:
        print("\nFailed to download the following links:")
        for link in failed_links:
            print(link)

# Download PDFs
download_pdfs(pcd_dataset)


  3%|▎         | 37/1210 [00:28<15:18,  1.28it/s]

Error downloading /articles/s42003-023-05133-1.pdf: Invalid URL '/articles/s42003-023-05133-1.pdf': No scheme supplied. Perhaps you meant https:///articles/s42003-023-05133-1.pdf?


  9%|▉         | 111/1210 [01:31<15:14,  1.20it/s]

Error downloading http://machinelearning.wustl.edu/mlpapers/paper_files/AISTATS2011_GlorotBB11.pdf: HTTPConnectionPool(host='machinelearning.wustl.edu', port=80): Max retries exceeded with url: /mlpapers/paper_files/AISTATS2011_GlorotBB11.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7c8a5272b160>: Failed to resolve 'machinelearning.wustl.edu' ([Errno -2] Name or service not known)"))


 14%|█▍        | 169/1210 [02:38<15:24,  1.13it/s]

...

Error downloading /articles/s41586-023-06735-9.pdf: Invalid URL '/articles/s41586-023-06735-9.pdf': No scheme supplied. Perhaps you meant https:///articles/s41586-023-06735-9.pdf?


 90%|████████▉ | 1084/1210 [18:59<01:17,  1.63it/s]

Error downloading /articles/s41586-023-06419-4.pdf: Invalid URL '/articles/s41586-023-06419-4.pdf': No scheme supplied. Perhaps you meant https:///articles/s41586-023-06419-4.pdf?


100%|██████████| 1210/1210 [20:49<00:00,  1.03s/it]


Failed to download the following links:
https://pubmed.ncbi.nlm.nih.gov/19000969/
https://huggingface.co/karakuri-ai/karakuri-lm-70b-v0.1
https://www.aclweb.org/anthology/D12-1110/
https://www.semanticscholar.org/paper/Statistical-language-modeling-using-the-toolkit-Clarkson-Rosenfeld/fdf4aa623e4d5b5edaeb873ed8e8b1cef0b59c87
https://mp.weixin.qq.com/s/BUQWZ5EdR19i40GuFofpBg
https://openai.com/blog/image-gpt/
https://ieeexplore.ieee.org/abstract/document/7913730
https://jpn.nec.com/press/202307/20230706_02.html
https://www.adept.ai/blog/adept-fuyu-heavy
https://www.anthropic.com/index/introducing-claude
https://link.springer.com/article/10.1007/BF00337288
https://openai.com/blog/language-unsupervised/
https://engineering.linecorp.com/ja/blog/3.6-billion-parameter-japanese-language-model
http://www.lrec-conf.org/proceedings/lrec2022/pdf/2022.lrec-1.376.pdf

https://academic.oup.com/comjnl/article/6/3/232/360077
https://ieeexplore.ieee.org/document/6032006
https://www.microsoft.com/en-us

# Convert PDFs to text

In [5]:
from google.colab import drive
from tqdm import tqdm
from PyPDF2 import PdfReader
import os

drive.mount('/content/drive')

def convert_pdfs_to_text(pdf_folder='pdfs/', text_folder='/content/drive/My Drive/TextPapers/'):
    """Convert PDFs in a folder to text files."""
    # Create the destination folder in Google Drive if it doesn't exist
    os.makedirs(text_folder, exist_ok=True)
    for pdf_file in tqdm(os.listdir(pdf_folder)):
        if pdf_file.endswith('.pdf'):
            try:
                reader = PdfReader(f"{pdf_folder}{pdf_file}")
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                text_file = pdf_file.replace('.pdf', '.txt')
                with open(f"{text_folder}{text_file}", 'w', encoding='utf-8') as f:
                    f.write(text)
            except Exception as e:
                print(f"Error processing {pdf_file}: {e}")

# Convert PDFs to text
convert_pdfs_to_text()

Mounted at /content/drive


  1%|          | 8/758 [00:12<16:13,  1.30s/it]

Error processing CODEFUSION (Python).pdf: EOF marker not found


  2%|▏         | 13/758 [00:18<21:59,  1.77s/it]

Error processing Context-dependent RNN.pdf: EOF marker not found


  2%|▏         | 17/758 [00:21<14:19,  1.16s/it]

Error processing Libratus.pdf: EOF marker not found


 63%|██████▎   | 477/758 [10:33<05:27,  1.16s/it]

Error processing RNN+LDA+KN5+cache.pdf: EOF marker not found


100%|██████████| 758/758 [16:08<00:00,  1.28s/it]


In [9]:
from tqdm import tqdm

def perform_keyword_analysis(text_folder='/content/drive/My Drive/TextPapers/'):
    """Perform keyword analysis on text files, using lemmatization to reduce words to their base form."""
    stop_words = set(stopwords.words('english')).union(additional_stopwords)
    keyword_counts = Counter()
    for text_file in tqdm(os.listdir(text_folder)):
        if text_file.endswith('.txt'):
            with open(f"{text_folder}{text_file}", 'r', encoding='utf-8') as f:
                text = f.read().lower()
                doc = nlp(text)
                lemmas = [token.lemma_ for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 1]
                keyword_counts.update(lemmas)
    total_keywords = len(keyword_counts)
    return keyword_counts.most_common(), total_keywords

keyword_analysis, total_keywords = perform_keyword_analysis()
print(f"Top 10 keywords: {keyword_analysis[:10]}")
print(f"Total number of unique keywords assessed: {total_keywords}")



100%|██████████| 754/754 [28:10<00:00,  2.24s/it]

Top 10 keywords: [('model', 78186), ('use', 42882), ('training', 28528), ('language', 27349), ('task', 23502), ('train', 20948), ('image', 19266), ('network', 19247), ('dataset', 18421), ('result', 18229)]
Total number of unique keywords assessed: 329163


# Keyword Extraction over time: new keywords that pop up

Question: What are the keywords that pop up or emerge?

Answer:


Total papers analyzed: 743

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install fuzzywuzzy[speedup]
import pandas as pd
import os
from collections import Counter
import spacy
from nltk.corpus import stopwords
import nltk
from fuzzywuzzy import process

nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

stop_words = set(stopwords.words('english')).union({'et', 'al'})

def perform_keyword_analysis(text):
    """does keyword analysis on text, using lemmatization and excluding stopwords."""
    nlp.max_length = len(text) + 10000
    doc = nlp(text)
    lemmas = [token.lemma_.lower() for token in doc if token.lemma_ not in stop_words and token.pos_ != 'PUNCT' and not token.lemma_.isdigit() and len(token.lemma_) > 1 and not token.lemma_.startswith('/')]
    lemma_freq = Counter(lemmas)
    return lemma_freq.most_common()

def find_best_match(name, choices, threshold=80):
    """gets the best match for a given name from a list of choices, with a minimum similarity threshold -- using to link text files with entry in dataset"""
    match = process.extractOne(name, choices)
    if match:
        best_match, score, _ = match
        if score >= threshold:
            return best_match
    return None

pcd_dataset = pd.read_csv("https://epochai.org/data/epochdb/all_systems.csv")

# Step 1: get publication year
pcd_dataset['Publication Year'] = pd.to_datetime(pcd_dataset['Publication date']).dt.year

# preprocess the System names in the dataset
processed_system_names = pcd_dataset['System'].str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_')

# Step 2: group text files by publication year
text_files_by_year = {}
text_folder = '/content/drive/My Drive/TextPapers/'
for text_file in os.listdir(text_folder):
    article_name = text_file.replace('.txt', '').replace('_', ' ')
    best_match = find_best_match(article_name, processed_system_names)
    if best_match is None:
        print(f"Warning: No matching entry found in dataset for text file '{text_file}'")
        continue
    year = pcd_dataset[processed_system_names == best_match]['Publication Year'].values[0]
    if year not in text_files_by_year:
        text_files_by_year[year] = []
    with open(f'{text_folder}{text_file}', 'r', encoding='utf-8') as f:
        text_content = f.read()
        text_files_by_year[year].append(text_content)

# Step 3: keyword analysis for each year
keywords_by_year = {}
papers_count_by_year = {}
for year, texts in text_files_by_year.items():
    all_text = ' '.join(texts)
    keywords_by_year[year] = perform_keyword_analysis(all_text)
    papers_count_by_year[year] = len(texts)

# Step 4: tracks emergence of new keywords
new_keywords_by_year = {}
all_keywords = set()
for year in sorted(keywords_by_year.keys()):
    current_year_keywords = set([keyword for keyword, _ in keywords_by_year[year]])
    new_keywords = current_year_keywords - all_keywords
    new_keywords_by_year[year] = new_keywords
    all_keywords.update(current_year_keywords)

output_file = '/content/drive/My Drive/NewKeywordsByYear.txt'
with open(output_file, 'w') as f:
    for year, new_keywords in new_keywords_by_year.items():
        f.write(f"Year: {year}, New Keywords: {', '.join(new_keywords)}\n")

total_papers_analyzed = sum(papers_count_by_year.values())
print(f"Total papers analyzed: {total_papers_analyzed}")

for year, count in papers_count_by_year.items():
    print(f"Year {year}: {count} papers analyzed")

# Print the top 5 new keywords for each year
for year, new_keywords in sorted(new_keywords_by_year.items()):
    keywords_freq_dict = dict(keywords_by_year[year])
    top_5_new_keywords = sorted(new_keywords, key=lambda k: keywords_freq_dict.get(k, 0), reverse=True)[:5]
    print(f"Year: {year}, Top 5 New Keywords: {', '.join(top_5_new_keywords)}")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Total papers analyzed: 745
Year 2019: 89 papers analyzed
Year 2020: 65 papers analyzed
Year 2023: 100 papers analyzed
Year 2018: 58 papers analyzed
Year 2014: 21 papers analyzed
Year 2022: 84 papers analyzed
Year 1999: 1 papers analyzed
Year 2021: 96 papers analyzed
Year 2007: 14 papers analyzed
Year 1997: 17 papers analyzed
Year 2016: 41 papers analyzed
Year 1960: 2 papers analyzed
Year 2012: 14 papers analyzed
Year 2013: 17 papers analyzed
Year 2017: 75 papers analyzed
Year 1992: 1 papers analyzed
Year 2009: 6 papers analyzed
Year 2015: 15 papers analyzed
Year 2003: 3 papers analyzed
Year 2011: 3 papers analyzed
Year 2008: 2 papers analyzed
Year 2024: 6 papers analyzed
Year 2006: 4 papers analyzed
Year 1998: 2 papers analyzed
Year 2004: 1 papers analyzed
Year 2010: 2 papers analyzed
Year 1950: 1 papers analyzed
Year 2002: 1 papers analyzed
Year 2001: 1 papers analyzed
Year 2000: 1 papers analyzed
Year 1986: 1 papers analyzed
Year 1967: 1 papers analyzed
Year: 1950, Top 5 New Keywords